# Training the Random Forest for Satellite Data Classification

Credit: Dr.Nitt TU Band 2020

A brief explanation of the RandomForest algorithm comes from the name. Rather than utilize the predictions of a single decision tree, the algorithm will take the ensemble result of a large number of decision trees (a forest of them). The "Random" part of the name comes from the term "bootstrap aggregating", or "bagging". What this means is that each tree within the forest only gets to train on some subset of the full training dataset (the subset is determined by sampling with replacement). The elements of the training data for each tree that are left unseen are held "out-of-bag" for estimation of accuracy. Randomness also helps decide which feature input variables are seen at each node in each decision tree. Once all individual trees are fit to the random subset of the training data, using a random set of feature variable at each node, the ensemble of them all is used to give the final prediction.

![](https://miro.medium.com/max/1350/1*j4TxWuLiRL-nmjQ89HZMBw.gif)

In [ ]:
from __future__ import print_function, division

import skimage.io as io
import numpy as np
import os, shutil

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.externals import joblib
import pandas as pd

from osgeo import gdal, gdal_array

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

## Preparing the dataset

In [ ]:
# set up directory with Image data**
rootdir = 'image/'
# set path to training data
path_pix = 'sample/'
# path to model
path_model = 'model/'
# path to classification result
path_class = 'class_out/'
# Path result with cmap
class_cmap = 'class_out_cmap/'

In [ ]:
def training():
	# path to image.tif
	raster = rootdir + 'ppnmulti4b_img.tif'
	# path to corresponding pixel samples (training data)
	samples = path_pix + 'ppn_sam.tif'
	
	# read in image raster
	img_ds = io.imread(raster)
	# convert to 16 bit numpy array
	img = np.array(img_ds, dtype='int16')
	
	# do the same with sample pixels
	roi_ds = io.imread(samples)
	roi = np.array(roi_ds, dtype='int16')
	
	# read in labels
	labels = np.unique(roi[roi>0])
	print('The training data include {n} classes: {classes}'.format(n=labels.size, classes=labels))
	
	# compose your X,Y data (dataset - training data)
	#X = img[roi>0]
	X = img[roi>0, :]
	Y = roi[roi>0]
	
	# assign class weight (class 1 has the weight 3, etc)
	#weights = {1:1, 2:2, 3:2, 4:3}
	#weights = {1:1, 2:3, 3:1, 4:3, 5:1, 6:1, 7:1}
	weights = {1:2, 2:2, 3:3, 5:1, 6:3, 7:1}
	
	# build random forest classifier
	#rf = RandomForestClassifier(class_weight=weights, n_estimators=200, criterion='gini',  max_depth=20, min_samples_split=4, min_samples_leaf=2, max_features='auto', bootstrap=True, oob_score=True, n_jobs=1, random_state=None, verbose=True)
	#rf = RandomForestClassifier(class_weight=weights, n_estimators=1000, criterion='gini',  max_depth=20, min_samples_split=4, min_samples_leaf=2, max_features='auto', bootstrap=True, oob_score=True, n_jobs=1, random_state=None, verbose=True)

	rf = RandomForestClassifier(class_weight=weights, n_estimators=200, criterion='gini', max_depth=20, min_samples_leaf=2, min_samples_split=4, max_features='auto', bootstrap=True, oob_score=True, n_jobs=1, random_state=None, verbose=True)

	# alternatively you may try out a Gradient Boosting Classifier 
    # It is much less RAM consuming and considers weak training data    
	#rf = GradientBoostingClassifier(n_estimators = 2000, min_samples_leaf = 1, min_samples_split = 4, max_depth = 4, max_features = 'auto', learning_rate = 0.1, subsample = 1, random_state = None, warm_start = True)

    # now fit training data with the original dataset
    #rf = rf.fit(X,Y)
	rf = rf.fit(X,Y)
	
	print('Our OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100))
	bands = [1, 2, 3, 4]

	for b, imp in zip(bands, rf.feature_importances_):
		print('Band {b} importance: {imp}'.format(b=b, imp=imp))

	# Setup a dataframe -- just like R
	df = pd.DataFrame()
	df['truth'] = Y
	df['predict'] = rf.predict(X)

	# Cross-tabulate predictions
	print(pd.crosstab(df['truth'], df['predict'], margins=True))
	
    # export Random Forest / Gradient Boosting Model        
	model = path_model + "model.pkl"
	joblib.dump(rf, model)
	
	return img

![](https://miro.medium.com/max/700/1*SjeRHukeJZYPOQPe2y-QuQ.jpeg)

In [ ]:
img = training()
print("Training Finished")

In [ ]:
def classification():
	# path to image.tif
	raster = rootdir + 'ppnmulti4b_img.tif'
	# path to corresponding pixel samples (training data)
	samples = path_pix + 'pnn_sam.tif'

    # Read worldfile of original dataset
	tfw_old = str(raster.split(".tif")[0]) + ".tfw"     

	# Read Data    
	img_ds = io.imread(raster)   
	img = np.array(img_ds, dtype='int16')    

    # call your random forest model
	rf = path_model + "model.pkl"   
	clf = joblib.load(rf)

    # Classification of array and save as image (3 refers to the number of multitemporal bands in the stack) 
	new_shape = (img.shape[0] * img.shape[1], img.shape[2])
	img_as_array = img[:, :, :4].reshape(new_shape)  # band 0, 1, 2 (1, 2, 3)
	class_prediction = clf.predict(img_as_array)
	class_prediction = class_prediction.reshape(img[:, :, 0].shape)

    # now export your classificaiton
	classification = path_class  + "classification.tif" 
	io.imsave(classification, class_prediction)    

    # Assign Worldfile to classified image    
	tfw_new = classification.split(".tif")[0] + ".tfw"   
	shutil.copy(tfw_old, tfw_new)

	return class_prediction

![](https://miro.medium.com/max/700/1*w5z0YYvpEBXINh4VUobxOg.gif)

In [ ]:
class_prediction = classification()
n = class_prediction.max()

In [ ]:
# First setup a 5-4-3 composite
def color_stretch(image, index, minmax=(0, 65536)):
    colors = image[:, :, index].astype(np.float64)

    max_val = minmax[1]
    min_val = minmax[0]

    # Enforce maximum and minimum values
    colors[colors[:, :, :] > max_val] = max_val
    colors[colors[:, :, :] < min_val] = min_val

    for b in range(colors.shape[2]):
        colors[:, :, b] = colors[:, :, b] * 1 / (max_val - min_val)
        
    return colors

![](https://miro.medium.com/max/700/1*DP57Bv0fS9oIWP2Bdvm4-A.jpeg)

In [ ]:
img321 = color_stretch(img, [3, 2, 1], (0, 15000))

In [ ]:
# Next setup a colormap for our map
colors = dict((
    (0, (0, 0, 0, 255)),  # Nodata
    (1, (0, 0, 255, 255)),  # Ra
    (2, (0, 0, 255, 255)),  # Rm
    (3, (0, 255, 255, 255)),  # Aa
    (4, (127, 0, 255, 255)),  # Bp
    (5, (102, 0, 102, 255)),  # Mx
	(6, (255, 0, 0, 255)), # Wa
	(7, (128, 128, 128, 255)) # Urban
))

In [ ]:
# Put 0 - 255 as float 0 - 1
for k in colors:
    v = colors[k]
    _v = [_v / 255.0 for _v in v]
    colors[k] = _v

In [ ]:
index_colors = [colors[key] if key in colors else 
                (255, 255, 255, 0) for key in range(1, n + 1)]
cmap = plt.matplotlib.colors.ListedColormap(index_colors, 'Classification', n)

print(index_colors)

In [ ]:
plt.subplot(121)
plt.imshow(img321)

plt.subplot(122)
plt.imshow(class_prediction, cmap=cmap, interpolation='none')

#Save classification
classification = class_cmap  + "classification_cmap.tif" 
io.imsave(classification, class_prediction)   

# tfw file for cmap result
raster = rootdir + 'ppnmulti4b_img.tif'
tfw_old = str(raster.split(".tif")[0]) + ".tfw"
tfw_new = classification.split(".tif")[0] + ".tfw"   
shutil.copy(tfw_old, tfw_new)

In [ ]:
print("Classification finished")

In [ ]:
plt.show()

![](https://miro.medium.com/max/700/1*Rg1Viw8hYkjmBlL04mTGwA.gif)